In [ ]:
import numpy as np
import os
import glob
import cv2
import random

In [ ]:
def padding(image,size):
    image = cv2.copyMakeBorder(image,size,size,size,size,cv2.BORDER_REFLECT_101)
    return image

In [ ]:
def balance_classes(image):

    vessels = cv2.countNonZero(image)
    backgr = image.size-vessels

    # Extract from the GT the indices of vessels/background to get balanced classes
    
    # Vessel pixels
    idx_vi, idx_vj = np.where(image !=0)
    idx_v =  list(zip(idx_vi, idx_vj))
    
    # Background pixels
    idx_bi, idx_bj = np.where(image ==0)
    idx_b = list(zip(idx_bi, idx_bj))

    
    #  Select the same number of vessel/background pixels
    random.seed(42)
    if vessels < backgr:
        idx_b = random.sample(idx_b, vessels)
        
    else:
        idx_v = random.sample(idx_v, backgr)
    
    return idx_v, idx_b

In [ ]:
def save_subimages(image, gt_im, file, pad_size,neig_size,idx_v, idx_b):
    
    list_ves = [(idx[0]+pad_size, idx[1]+pad_size) for idx in idx_v]
    list_bac = [(idx[0]+pad_size, idx[1]+pad_size) for idx in idx_b]
    for im_n, idx in enumerate(list_ves):
        crop_im = (image[idx[0]-pad_size:idx[0]+(pad_size+1) , idx[1]-pad_size:idx[1]+(pad_size+1)])
        new_file =  file+ '%05d' %(im_n)
        #cv2.imwrite('D:/CNN/test_im/' + new_file +'.png',crop_im)
        cv2.imwrite('./training_all/vessel/' + new_file +'.png',crop_im)
    for im_n, idx in enumerate(list_bac):
        crop_im = (image[idx[0]-pad_size:idx[0]+(pad_size+1) , idx[1]-pad_size:idx[1]+(pad_size+1)])
        new_file =  file+ '%05d' %(im_n)
        #cv2.imwrite('D:/CNN/test_im/' + new_file +'.png',crop_im)
        cv2.imwrite('./training_all/background/' + new_file +'.png',crop_im)
    return  

In [ ]:
# Set image paths
images_path = './training_im'
gt_path = "./gt_training"

In [ ]:
# Create output folders
import os
training_path =  './training_all'
vessel_path = './training_all/vessel'
background_path = './training_all/background'
os.mkdir(training_path);
os.mkdir(vessel_path);
os.mkdir(background_path);

In [ ]:
# Define padding size and the neighborhood size
pad_size = 30
neig_size = 61

y = list()
for num, infile in enumerate(glob.glob(images_path +'/*.tif')):
    p, im_name = os.path.split(infile)
    file, ext = os.path.splitext(im_name)
    print(file)
    # Read Ground Truth
    gt = cv2.imread(gt_path +"/" + file + ".png",0)
    print(gt.shape)
    im = cv2.imread(infile,0) # because they are gray scale
    print(im.shape)
    print(im.size)
    
    # Extract vessel/background pixels
    idx_v, idx_b = balance_classes(gt)
    
    # Padding
    pad_im = padding(im, 30)
    save_subimages(pad_im, gt, file, pad_size, neig_size, idx_v, idx_b)
    